In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.set_option('display.max_row',500)
pd.set_option('display.max_columns',500)

In [2]:
train=pd.read_csv('dataset/train.csv')
test=pd.read_csv('dataset/test.csv')

df_train=pd.concat([train,test])

In [107]:
print(train.shape)

(1138136, 51)


In [3]:
df_train.info(null_counts=True)

C:\Users\SYU\AppData\Local\Temp\ipykernel_7908\116129542.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_train.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2845342 entries, 0 to 1707205
Data columns (total 47 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     2845342 non-null  object 
 1   Start_Time             2845342 non-null  object 
 2   End_Time               2845342 non-null  object 
 3   Start_Lat              2845342 non-null  float64
 4   Start_Lng              2845342 non-null  float64
 5   End_Lat                2845342 non-null  float64
 6   End_Lng                2845342 non-null  float64
 7   Distance(mi)           2845342 non-null  float64
 8   Description            2845342 non-null  object 
 9   Number                 1101431 non-null  float64
 10  Street                 2845340 non-null  object 
 11  Side                   2845342 non-null  object 
 12  City                   2845205 non-null  object 
 13  County                 2845342 non-null  object 
 14  State             

In [4]:
bool_list=df_train_num=df_train.select_dtypes(bool).columns
df_train[bool_list]=df_train[bool_list].astype(int)

df_train

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity
0,A-462285,2021-12-15 18:11:00.000000000,2021-12-15 20:18:24.000000000,36.059036,-95.922034,36.060756,-95.921231,0.127,Slow traffic from E 73rd St to S Braden Ave du...,7181.0,S Yale Ave,L,Tulsa,Tulsa,OK,74136-6326,US,US/Central,KRVS,2021-12-15 17:53:00,76.0,76.0,58.0,29.06,10.0,SSW,23.0,0.00,Mostly Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Day,2.0
1,A-2327376,2020-02-04 17:57:00,2020-02-04 18:28:02,33.913961,-118.107838,33.913961,-118.107838,0.000,At I-605 - Accident.,NaN,Glenn Anderson Fwy W,R,Norwalk,Los Angeles,CA,90650,US,US/Pacific,KFUL,2020-02-04 17:53:00,56.0,56.0,17.0,30.00,10.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Day,Day,2.0
2,A-1276216,2021-12-06 18:26:00.000000000,2021-12-06 18:40:30.000000000,46.154347,-107.435978,46.172417,-107.289048,7.141,Slow traffic on I-94 E from Tullock Rd/Exit 53...,NaN,I-94 E,R,Bighorn,Treasure,MT,59010,US,US/Mountain,KM46,2021-12-06 18:26:00,20.0,10.0,85.0,27.12,1.0,NNE,8.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,2.0
3,A-2692390,2018-02-17 17:23:05,2018-02-17 23:23:05,40.647743,-76.419335,40.621655,-76.441113,2.134,At US-209/Exit 107 - Accident. Traffic problem.,NaN,I-81 S,R,Tremont,Schuylkill,PA,17981,US,US/Eastern,KMUI,2018-02-17 17:22:00,30.2,25.4,100.0,30.21,0.5,SE,4.6,NaN,Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,4.0
4,A-2357636,2020-01-01 20:38:00,2020-01-01 21:32:27,33.935228,-118.092480,33.935228,-118.092480,0.000,At Florence Ave/Exit 123 - Accident.,NaN,I-5 N,R,Downey,Los Angeles,CA,90241,US,US/Pacific,KLGB,2020-01-01 20:53:00,56.0,56.0,84.0,29.87,8.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,A-2775688,2019-04-10 17:17:47,2019-04-10 17:44:48,41.876190,-87.657030,41.876150,-87.660860,0.197,At Racine Ave/Exit 29A - Accident.,NaN,S Racine Ave,R,Chicago,Cook,IL,60607,US,US/Central,KMDW,2019-04-10 16:53:00,41.0,31.0,79.0,29.26,10.0,ENE,22.0,0.01,Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN
1707202,A-856779,2021-11-17 16:10:00,2021-11-17 18:38:45,34.025207,-118.433563,34.026413,-118.431066,0.165,Slow traffic from S Barrington Ave (National B...,11420.0,National Blvd,L,Los Angeles,Los Angeles,CA,90064-3729,US,US/Pacific,KSMO,2021-11-17 15:51:00,61.0,61.0,78.0,29.83,10.0,WSW,8.0,0.00,Fair,0,0,1,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN
1707203,A-194577,2017-06-01 09:55:57,2017-06-01 15:55:57,42.617250,-71.291750,42.619610,-71.285630,0.351,At Woburn St/Exit 37 - Accident.,NaN,I-495 N,R,Lowell,Middlesex,MA,01852,US,US/Eastern,KLWM,2017-06-01 09:54:00,70.0,NaN,63.0,29.90,10.0,Variable,5.8,NaN,Clear,0,0,0,0,1,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN
1707204,A-594123,2021-12-04 22:22:05,2021-12-05 02:49:21,44.948605,-92.959087,44.948566,-92.956429,0.130,Construction on I-94 near I-694 Drive with cau...,NaN,I-494 W,R,Saint Paul,Washington,MN,55125,US,US/Central,KSTP,2021-12-04 22:29:00,30.0,24.0,88.0,29.44,1.0,SE,6.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,NaN


In [5]:

df_train['Start_Time']=pd.to_datetime(df_train['Start_Time'])
df_train['End_Time']=pd.to_datetime(df_train['End_Time'])

df_train['st_Year']=df_train['Start_Time'].dt.year
df_train['st_Month']=df_train['Start_Time'].dt.month
df_train['st_Day']=df_train['Start_Time'].dt.day
df_train['st_Weekday']=df_train['Start_Time'].dt.day_name()
df_train['st_Hour']=df_train['Start_Time'].dt.hour
df_train['st_Minutes']=df_train['Start_Time'].dt.minute
df_train['st_Seconds']=df_train['Start_Time'].dt.second

df_train['end_Year']=df_train['End_Time'].dt.year
df_train['end_Month']=df_train['End_Time'].dt.month
df_train['end_Day']=df_train['End_Time'].dt.day
df_train['end_Weekday']=df_train['End_Time'].dt.day_name()
df_train['end_Hour']=df_train['End_Time'].dt.hour
df_train['end_Minutes']=df_train['End_Time'].dt.minute
df_train['end_Seconds']=df_train['End_Time'].dt.second

df_train

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,st_Year,st_Month,st_Day,st_Weekday,st_Hour,st_Minutes,st_Seconds,end_Year,end_Month,end_Day,end_Weekday,end_Hour,end_Minutes,end_Seconds
0,A-462285,2021-12-15 18:11:00,2021-12-15 20:18:24,36.059036,-95.922034,36.060756,-95.921231,0.127,Slow traffic from E 73rd St to S Braden Ave du...,7181.0,S Yale Ave,L,Tulsa,Tulsa,OK,74136-6326,US,US/Central,KRVS,2021-12-15 17:53:00,76.0,76.0,58.0,29.06,10.0,SSW,23.0,0.00,Mostly Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Day,2.0,2021,12,15,Wednesday,18,11,0,2021,12,15,Wednesday,20,18,24
1,A-2327376,2020-02-04 17:57:00,2020-02-04 18:28:02,33.913961,-118.107838,33.913961,-118.107838,0.000,At I-605 - Accident.,NaN,Glenn Anderson Fwy W,R,Norwalk,Los Angeles,CA,90650,US,US/Pacific,KFUL,2020-02-04 17:53:00,56.0,56.0,17.0,30.00,10.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Day,Day,2.0,2020,2,4,Tuesday,17,57,0,2020,2,4,Tuesday,18,28,2
2,A-1276216,2021-12-06 18:26:00,2021-12-06 18:40:30,46.154347,-107.435978,46.172417,-107.289048,7.141,Slow traffic on I-94 E from Tullock Rd/Exit 53...,NaN,I-94 E,R,Bighorn,Treasure,MT,59010,US,US/Mountain,KM46,2021-12-06 18:26:00,20.0,10.0,85.0,27.12,1.0,NNE,8.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,2.0,2021,12,6,Monday,18,26,0,2021,12,6,Monday,18,40,30
3,A-2692390,2018-02-17 17:23:05,2018-02-17 23:23:05,40.647743,-76.419335,40.621655,-76.441113,2.134,At US-209/Exit 107 - Accident. Traffic problem.,NaN,I-81 S,R,Tremont,Schuylkill,PA,17981,US,US/Eastern,KMUI,2018-02-17 17:22:00,30.2,25.4,100.0,30.21,0.5,SE,4.6,NaN,Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,4.0,2018,2,17,Saturday,17,23,5,2018,2,17,Saturday,23,23,5
4,A-2357636,2020-01-01 20:38:00,2020-01-01 21:32:27,33.935228,-118.092480,33.935228,-118.092480,0.000,At Florence Ave/Exit 123 - Accident.,NaN,I-5 N,R,Downey,Los Angeles,CA,90241,US,US/Pacific,KLGB,2020-01-01 20:53:00,56.0,56.0,84.0,29.87,8.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,2.0,2020,1,1,Wednesday,20,38,0,2020,1,1,Wednesday,21,32,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,A-2775688,2019-04-10 17:17:47,2019-04-10 17:44:48,41.876190,-87.657030,41.876150,-87.660860,0.197,At Racine Ave/Exit 29A - Accident.,NaN,S Racine Ave,R,Chicago,Cook,IL,60607,US,US/Central,KMDW,2019-04-10 16:53:00,41.0,31.0,79.0,29.26,10.0,ENE,22.0,0.01,Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN,2019,4,10,Wednesday,17,17,47,2019,4,10,Wednesday,17,44,48
1707202,A-856779,2021-11-17 16:10:00,2021-11-17 18:38:45,34.025207,-118.433563,34.026413,-118.431066,0.165,Slow traffic from S Barrington Ave (National B...,11420.0,National Blvd,L,Los Angeles,Los Angeles,CA,90064-3729,US,US/Pacific,KSMO,2021-11-17 15:51:00,61.0,61.0,78.0,29.83,10.0,WSW,8.0,0.00,Fair,0,0,1,0,0,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN,2021,11,17,Wednesday,16,10,0,2021,11,17,Wednesday,18,38,45
1707203,A-194577,2017-06-01 09:55:57,2017-06-01 15:55:57,42.617250,-71.291750,42.619610,-71.285630,0.351,At Woburn St/Exit 37 - Accident.,NaN,I-495 N,R,Lowell,Middlesex,MA,01852,US,US/Eastern,KLWM,2017-06-01 09:54:00,70.0,NaN,63.0,29.90,10.0,Variable,5.8,NaN,Clear,0,0,0,0,1,0,0,0,0,0,0,0,0,Day,Day,Day,Day,NaN,2017,6,1,Thursday,9,55,57,2017,6,1,Thursday,15,55,57
1707204,A-594123,2021-12-04 22:22

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2845342 entries, 0 to 1707205
Data columns (total 61 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Start_Time             datetime64[ns]
 2   End_Time               datetime64[ns]
 3   Start_Lat              float64       
 4   Start_Lng              float64       
 5   End_Lat                float64       
 6   End_Lng                float64       
 7   Distance(mi)           float64       
 8   Description            object        
 9   Number                 float64       
 10  Street                 object        
 11  Side                   object        
 12  City                   object        
 13  County                 object        
 14  State                  object        
 15  Zipcode                object        
 16  Country                object        
 17  Timezone               object        
 18  Airport_Code          

In [7]:
daynight=df_train.iloc[:,42:46].columns
df_train[daynight]=df_train[daynight].replace({'Night':1, 'Day':0})

df_train

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,st_Year,st_Month,st_Day,st_Weekday,st_Hour,st_Minutes,st_Seconds,end_Year,end_Month,end_Day,end_Weekday,end_Hour,end_Minutes,end_Seconds
0,A-462285,2021-12-15 18:11:00,2021-12-15 20:18:24,36.059036,-95.922034,36.060756,-95.921231,0.127,Slow traffic from E 73rd St to S Braden Ave du...,7181.0,S Yale Ave,L,Tulsa,Tulsa,OK,74136-6326,US,US/Central,KRVS,2021-12-15 17:53:00,76.0,76.0,58.0,29.06,10.0,SSW,23.0,0.00,Mostly Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,0.0,2.0,2021,12,15,Wednesday,18,11,0,2021,12,15,Wednesday,20,18,24
1,A-2327376,2020-02-04 17:57:00,2020-02-04 18:28:02,33.913961,-118.107838,33.913961,-118.107838,0.000,At I-605 - Accident.,NaN,Glenn Anderson Fwy W,R,Norwalk,Los Angeles,CA,90650,US,US/Pacific,KFUL,2020-02-04 17:53:00,56.0,56.0,17.0,30.00,10.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,2.0,2020,2,4,Tuesday,17,57,0,2020,2,4,Tuesday,18,28,2
2,A-1276216,2021-12-06 18:26:00,2021-12-06 18:40:30,46.154347,-107.435978,46.172417,-107.289048,7.141,Slow traffic on I-94 E from Tullock Rd/Exit 53...,NaN,I-94 E,R,Bighorn,Treasure,MT,59010,US,US/Mountain,KM46,2021-12-06 18:26:00,20.0,10.0,85.0,27.12,1.0,NNE,8.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2021,12,6,Monday,18,26,0,2021,12,6,Monday,18,40,30
3,A-2692390,2018-02-17 17:23:05,2018-02-17 23:23:05,40.647743,-76.419335,40.621655,-76.441113,2.134,At US-209/Exit 107 - Accident. Traffic problem.,NaN,I-81 S,R,Tremont,Schuylkill,PA,17981,US,US/Eastern,KMUI,2018-02-17 17:22:00,30.2,25.4,100.0,30.21,0.5,SE,4.6,NaN,Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.0,2018,2,17,Saturday,17,23,5,2018,2,17,Saturday,23,23,5
4,A-2357636,2020-01-01 20:38:00,2020-01-01 21:32:27,33.935228,-118.092480,33.935228,-118.092480,0.000,At Florence Ave/Exit 123 - Accident.,NaN,I-5 N,R,Downey,Los Angeles,CA,90241,US,US/Pacific,KLGB,2020-01-01 20:53:00,56.0,56.0,84.0,29.87,8.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2020,1,1,Wednesday,20,38,0,2020,1,1,Wednesday,21,32,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,A-2775688,2019-04-10 17:17:47,2019-04-10 17:44:48,41.876190,-87.657030,41.876150,-87.660860,0.197,At Racine Ave/Exit 29A - Accident.,NaN,S Racine Ave,R,Chicago,Cook,IL,60607,US,US/Central,KMDW,2019-04-10 16:53:00,41.0,31.0,79.0,29.26,10.0,ENE,22.0,0.01,Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2019,4,10,Wednesday,17,17,47,2019,4,10,Wednesday,17,44,48
1707202,A-856779,2021-11-17 16:10:00,2021-11-17 18:38:45,34.025207,-118.433563,34.026413,-118.431066,0.165,Slow traffic from S Barrington Ave (National B...,11420.0,National Blvd,L,Los Angeles,Los Angeles,CA,90064-3729,US,US/Pacific,KSMO,2021-11-17 15:51:00,61.0,61.0,78.0,29.83,10.0,WSW,8.0,0.00,Fair,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2021,11,17,Wednesday,16,10,0,2021,11,17,Wednesday,18,38,45
1707203,A-194577,2017-06-01 09:55:57,2017-06-01 15:55:57,42.617250,-71.291750,42.619610,-71.285630,0.351,At Woburn St/Exit 37 - Accident.,NaN,I-495 N,R,Lowell,Middlesex,MA,01852,US,US/Eastern,KLWM,2017-06-01 09:54:00,70.0,NaN,63.0,29.90,10.0,Variable,5.8,NaN,Clear,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2017,6,1,Thursday,9,55,57,2017,6,1,Thursday,15,55,57
1707204,A-594123,2021-12-04 22:22:05,2021-12-05 02:49:21,44

In [8]:
#df_train['Description']=df_train['Description'].apply(lambda x: pd.factorize(x)[0])
#df_train['Description']

In [9]:
df_train

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,st_Year,st_Month,st_Day,st_Weekday,st_Hour,st_Minutes,st_Seconds,end_Year,end_Month,end_Day,end_Weekday,end_Hour,end_Minutes,end_Seconds
0,A-462285,2021-12-15 18:11:00,2021-12-15 20:18:24,36.059036,-95.922034,36.060756,-95.921231,0.127,Slow traffic from E 73rd St to S Braden Ave du...,7181.0,S Yale Ave,L,Tulsa,Tulsa,OK,74136-6326,US,US/Central,KRVS,2021-12-15 17:53:00,76.0,76.0,58.0,29.06,10.0,SSW,23.0,0.00,Mostly Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,0.0,2.0,2021,12,15,Wednesday,18,11,0,2021,12,15,Wednesday,20,18,24
1,A-2327376,2020-02-04 17:57:00,2020-02-04 18:28:02,33.913961,-118.107838,33.913961,-118.107838,0.000,At I-605 - Accident.,NaN,Glenn Anderson Fwy W,R,Norwalk,Los Angeles,CA,90650,US,US/Pacific,KFUL,2020-02-04 17:53:00,56.0,56.0,17.0,30.00,10.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,2.0,2020,2,4,Tuesday,17,57,0,2020,2,4,Tuesday,18,28,2
2,A-1276216,2021-12-06 18:26:00,2021-12-06 18:40:30,46.154347,-107.435978,46.172417,-107.289048,7.141,Slow traffic on I-94 E from Tullock Rd/Exit 53...,NaN,I-94 E,R,Bighorn,Treasure,MT,59010,US,US/Mountain,KM46,2021-12-06 18:26:00,20.0,10.0,85.0,27.12,1.0,NNE,8.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2021,12,6,Monday,18,26,0,2021,12,6,Monday,18,40,30
3,A-2692390,2018-02-17 17:23:05,2018-02-17 23:23:05,40.647743,-76.419335,40.621655,-76.441113,2.134,At US-209/Exit 107 - Accident. Traffic problem.,NaN,I-81 S,R,Tremont,Schuylkill,PA,17981,US,US/Eastern,KMUI,2018-02-17 17:22:00,30.2,25.4,100.0,30.21,0.5,SE,4.6,NaN,Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.0,2018,2,17,Saturday,17,23,5,2018,2,17,Saturday,23,23,5
4,A-2357636,2020-01-01 20:38:00,2020-01-01 21:32:27,33.935228,-118.092480,33.935228,-118.092480,0.000,At Florence Ave/Exit 123 - Accident.,NaN,I-5 N,R,Downey,Los Angeles,CA,90241,US,US/Pacific,KLGB,2020-01-01 20:53:00,56.0,56.0,84.0,29.87,8.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2020,1,1,Wednesday,20,38,0,2020,1,1,Wednesday,21,32,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,A-2775688,2019-04-10 17:17:47,2019-04-10 17:44:48,41.876190,-87.657030,41.876150,-87.660860,0.197,At Racine Ave/Exit 29A - Accident.,NaN,S Racine Ave,R,Chicago,Cook,IL,60607,US,US/Central,KMDW,2019-04-10 16:53:00,41.0,31.0,79.0,29.26,10.0,ENE,22.0,0.01,Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2019,4,10,Wednesday,17,17,47,2019,4,10,Wednesday,17,44,48
1707202,A-856779,2021-11-17 16:10:00,2021-11-17 18:38:45,34.025207,-118.433563,34.026413,-118.431066,0.165,Slow traffic from S Barrington Ave (National B...,11420.0,National Blvd,L,Los Angeles,Los Angeles,CA,90064-3729,US,US/Pacific,KSMO,2021-11-17 15:51:00,61.0,61.0,78.0,29.83,10.0,WSW,8.0,0.00,Fair,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2021,11,17,Wednesday,16,10,0,2021,11,17,Wednesday,18,38,45
1707203,A-194577,2017-06-01 09:55:57,2017-06-01 15:55:57,42.617250,-71.291750,42.619610,-71.285630,0.351,At Woburn St/Exit 37 - Accident.,NaN,I-495 N,R,Lowell,Middlesex,MA,01852,US,US/Eastern,KLWM,2017-06-01 09:54:00,70.0,NaN,63.0,29.90,10.0,Variable,5.8,NaN,Clear,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2017,6,1,Thursday,9,55,57,2017,6,1,Thursday,15,55,57
1707204,A-594123,2021-12-04 22:22:05,2021-12-05 02:49:21,44

In [10]:
df_train['Description']=df_train['Description'].factorize()[0]
df_train

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,st_Year,st_Month,st_Day,st_Weekday,st_Hour,st_Minutes,st_Seconds,end_Year,end_Month,end_Day,end_Weekday,end_Hour,end_Minutes,end_Seconds
0,A-462285,2021-12-15 18:11:00,2021-12-15 20:18:24,36.059036,-95.922034,36.060756,-95.921231,0.127,0,7181.0,S Yale Ave,L,Tulsa,Tulsa,OK,74136-6326,US,US/Central,KRVS,2021-12-15 17:53:00,76.0,76.0,58.0,29.06,10.0,SSW,23.0,0.00,Mostly Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,0.0,2.0,2021,12,15,Wednesday,18,11,0,2021,12,15,Wednesday,20,18,24
1,A-2327376,2020-02-04 17:57:00,2020-02-04 18:28:02,33.913961,-118.107838,33.913961,-118.107838,0.000,1,NaN,Glenn Anderson Fwy W,R,Norwalk,Los Angeles,CA,90650,US,US/Pacific,KFUL,2020-02-04 17:53:00,56.0,56.0,17.0,30.00,10.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,2.0,2020,2,4,Tuesday,17,57,0,2020,2,4,Tuesday,18,28,2
2,A-1276216,2021-12-06 18:26:00,2021-12-06 18:40:30,46.154347,-107.435978,46.172417,-107.289048,7.141,2,NaN,I-94 E,R,Bighorn,Treasure,MT,59010,US,US/Mountain,KM46,2021-12-06 18:26:00,20.0,10.0,85.0,27.12,1.0,NNE,8.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2021,12,6,Monday,18,26,0,2021,12,6,Monday,18,40,30
3,A-2692390,2018-02-17 17:23:05,2018-02-17 23:23:05,40.647743,-76.419335,40.621655,-76.441113,2.134,3,NaN,I-81 S,R,Tremont,Schuylkill,PA,17981,US,US/Eastern,KMUI,2018-02-17 17:22:00,30.2,25.4,100.0,30.21,0.5,SE,4.6,NaN,Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.0,2018,2,17,Saturday,17,23,5,2018,2,17,Saturday,23,23,5
4,A-2357636,2020-01-01 20:38:00,2020-01-01 21:32:27,33.935228,-118.092480,33.935228,-118.092480,0.000,4,NaN,I-5 N,R,Downey,Los Angeles,CA,90241,US,US/Pacific,KLGB,2020-01-01 20:53:00,56.0,56.0,84.0,29.87,8.0,CALM,0.0,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.0,2020,1,1,Wednesday,20,38,0,2020,1,1,Wednesday,21,32,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,A-2775688,2019-04-10 17:17:47,2019-04-10 17:44:48,41.876190,-87.657030,41.876150,-87.660860,0.197,3471,NaN,S Racine Ave,R,Chicago,Cook,IL,60607,US,US/Central,KMDW,2019-04-10 16:53:00,41.0,31.0,79.0,29.26,10.0,ENE,22.0,0.01,Cloudy / Windy,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2019,4,10,Wednesday,17,17,47,2019,4,10,Wednesday,17,44,48
1707202,A-856779,2021-11-17 16:10:00,2021-11-17 18:38:45,34.025207,-118.433563,34.026413,-118.431066,0.165,55461,11420.0,National Blvd,L,Los Angeles,Los Angeles,CA,90064-3729,US,US/Pacific,KSMO,2021-11-17 15:51:00,61.0,61.0,78.0,29.83,10.0,WSW,8.0,0.00,Fair,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2021,11,17,Wednesday,16,10,0,2021,11,17,Wednesday,18,38,45
1707203,A-194577,2017-06-01 09:55:57,2017-06-01 15:55:57,42.617250,-71.291750,42.619610,-71.285630,0.351,243755,NaN,I-495 N,R,Lowell,Middlesex,MA,01852,US,US/Eastern,KLWM,2017-06-01 09:54:00,70.0,NaN,63.0,29.90,10.0,Variable,5.8,NaN,Clear,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,2017,6,1,Thursday,9,55,57,2017,6,1,Thursday,15,55,57
1707204,A-594123,2021-12-04 22:22:05,2021-12-05 02:49:21,44.948605,-92.959087,44.948566,-92.956429,0.130,464730,NaN,I-494 W,R,Saint Paul,Washington,MN,55125,US,US/Central,KSTP,2021-12-04 22:29:00,30.0,24.0,88.0,29.44,1.0,SE,6.0,0.00,Light Snow,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,NaN,2021,12,4,Saturday,22,22,5,2021,12,5,Sunday,2,49,21


In [11]:
df_train=df_train.drop(
    [ 'Start_Time', 'End_Time', 'Weather_Timestamp', 
    'Number', 'ID', 'Description', 'Zipcode',
    'Wind_Chill(F)', 'Precipitation(in)', 'Wind_Speed(mph)'],axis=1)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2845342 entries, 0 to 1707205
Data columns (total 51 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Start_Lat              float64
 1   Start_Lng              float64
 2   End_Lat                float64
 3   End_Lng                float64
 4   Distance(mi)           float64
 5   Street                 object 
 6   Side                   object 
 7   City                   object 
 8   County                 object 
 9   State                  object 
 10  Country                object 
 11  Timezone               object 
 12  Airport_Code           object 
 13  Temperature(F)         float64
 14  Humidity(%)            float64
 15  Pressure(in)           float64
 16  Visibility(mi)         float64
 17  Wind_Direction         object 
 18  Weather_Condition      object 
 19  Amenity                int32  
 20  Bump                   int32  
 21  Crossing               int32  
 22  Give_Way          

# 수치 범주형 데이터 처리
## 범주

In [75]:
df_train_cat=df_train.select_dtypes(object).columns
df_train_cat
df_train[df_train_cat]=df_train[df_train_cat].astype('category')

In [76]:
cat_list = df_train.isnull().sum().sort_values(ascending=False)>0
cat_list = cat_list[cat_list==True].index
df_train[cat_list] = df_train[cat_list].apply(lambda x: x.fillna(x.mode()[0]))

In [94]:
df_train_cat=df_train.select_dtypes('category').columns
df_train_cat
df_train[df_train_cat] = df_train[df_train_cat].apply(lambda x: pd.factorize(x)[0])

## 수치

In [95]:
n=['int32','int64','float64']
df_train_num=df_train.select_dtypes(include=n).columns

df_train[df_train_num]=df_train[df_train_num].apply(lambda x: x.fillna(x.mean()))

In [96]:
df_train

,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Street,Side,City,County,State,Country,Timezone,Airport_Code,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,st_Year,st_Month,st_Day,st_Weekday,st_Hour,st_Minutes,st_Seconds,end_Year,end_Month,end_Day,end_Weekday,end_Hour,end_Minutes,end_Seconds
0,36.059036,-95.922034,36.060756,-95.921231,0.127,0,0,0,0,0,0,0,0,76.0,58.0,29.06,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,0.0,2.000000,2021,12,15,0,18,11,0,2021,12,15,0,20,18,24
1,33.913961,-118.107838,33.913961,-118.107838,0.000,1,1,1,1,1,0,1,1,56.0,17.0,30.00,10.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,2.000000,2020,2,4,1,17,57,0,2020,2,4,1,18,28,2
2,46.154347,-107.435978,46.172417,-107.289048,7.141,2,1,2,2,2,0,2,2,20.0,85.0,27.12,1.0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.000000,2021,12,6,2,18,26,0,2021,12,6,2,18,40,30
3,40.647743,-76.419335,40.621655,-76.441113,2.134,3,1,3,3,3,0,3,3,30.2,100.0,30.21,0.5,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,4.000000,2018,2,17,3,17,23,5,2018,2,17,3,23,23,5
4,33.935228,-118.092480,33.935228,-118.092480,0.000,4,1,4,1,1,0,1,4,56.0,84.0,29.87,8.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.000000,2020,1,1,0,20,38,0,2020,1,1,0,21,32,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707201,41.876190,-87.657030,41.876150,-87.660860,0.197,7644,1,20,17,11,0,0,21,41.0,79.0,29.26,10.0,7,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.137571,2019,4,10,0,17,17,47,2019,4,10,0,17,44,48
1707202,34.025207,-118.433563,34.026413,-118.431066,0.165,6313,0,11,1,1,0,1,11,61.0,78.0,29.83,10.0,21,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.137571,2021,11,17,0,16,10,0,2021,11,17,0,18,38,45
1707203,42.617250,-71.291750,42.619610,-71.285630,0.351,3102,1,1220,226,31,0,3,695,70.0,63.0,29.90,10.0,11,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.137571,2017,6,1,6,9,55,57,2017,6,1,6,15,55,57
1707204,44.948605,-92.959087,44.948566,-92.956429,0.130,557,1,18,8,9,0,0,18,30.0,88.0,29.44,1.0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,2.137571,2021,12,4,3,22,22,5,2021,12,5,4,2,49,21


In [97]:
df_train.info(null_counts=True)

C:\Users\SYU\AppData\Local\Temp\ipykernel_7908\116129542.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_train.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2845342 entries, 0 to 1707205
Data columns (total 51 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Start_Lat              2845342 non-null  float64
 1   Start_Lng              2845342 non-null  float64
 2   End_Lat                2845342 non-null  float64
 3   End_Lng                2845342 non-null  float64
 4   Distance(mi)           2845342 non-null  float64
 5   Street                 2845342 non-null  int64  
 6   Side                   2845342 non-null  int64  
 7   City                   2845342 non-null  int64  
 8   County                 2845342 non-null  int64  
 9   State                  2845342 non-null  int64  
 10  Country                2845342 non-null  int64  
 11  Timezone               2845342 non-null  int64  
 12  Airport_Code           2845342 non-null  int64  
 13  Temperature(F)         2845342 non-null  float64
 14  Humidity(%)       

In [133]:
len(df_train)
train = df_train[:len(train)]
test = df_train[len(train):]

train.to_csv('Last_train.csv',index=False)
test.to_csv('Last_test.csv',index=False)

In [134]:
x_train=pd.read_csv('Last_train.csv')
x_test=pd.read_csv('Last_test.csv')
y=x_train['Severity']
X=x_train.drop('Severity',axis=1)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2, random_state=42,shuffle=True)

In [136]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(910508, 50) (227628, 50) (910508,) (227628,)


In [137]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138136 entries, 0 to 1138135
Data columns (total 50 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Start_Lat              1138136 non-null  float64
 1   Start_Lng              1138136 non-null  float64
 2   End_Lat                1138136 non-null  float64
 3   End_Lng                1138136 non-null  float64
 4   Distance(mi)           1138136 non-null  float64
 5   Street                 1138136 non-null  int64  
 6   Side                   1138136 non-null  int64  
 7   City                   1138136 non-null  int64  
 8   County                 1138136 non-null  int64  
 9   State                  1138136 non-null  int64  
 10  Country                1138136 non-null  int64  
 11  Timezone               1138136 non-null  int64  
 12  Airport_Code           1138136 non-null  int64  
 13  Temperature(F)         1138136 non-null  float64
 14  Humidity(%)       

In [138]:
x_train.isnull().sum().sum()

0

In [139]:
rf_model=RandomForestClassifier(n_jobs=-1)

rf_model.fit(X_train,y_train)
print(rf_model.predict(X_test))
print(y_test.values)

[2. 2. 2. ... 2. 2. 2.]
[2. 2. 2. ... 2. 2. 2.]


In [141]:
y_pred=rf_model.predict(x_test)
print("학습 데이터셋 정학도:",accuracy_score(rf_model.predict(x_train),y_train))
print("학습 데이터셋 정학도:",accuracy_score(y_test,y_pred))

C:\Users\SYU\miniconda3\envs\dacon\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Severity

  warnings.warn(message, FutureWarning)


ValueError: X has 51 features, but RandomForestClassifier is expecting 50 features as input.